In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import utilities


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
tickers = pd.read_csv('../data_cleaning/stocks/dense_95.csv')

In [ ]:
tickers["ticker"].values

In [ ]:
# count the days in each stock

def unique_days(ticker):
    df = utilities.load_stock(ticker)
    days = utilities.count_days(df)
    
    return days

unique_days("AAPL")